In [1]:
import os
os.chdir('..')

In [51]:
from models.encoders import MNISTEncoder
from models.decoders import MNISTDecoder
import torch
from torch.distributions import Normal, Bernoulli, kl_divergence

In [32]:
decoder = MNISTDecoder()

In [48]:
x = torch.randn(32, 1, 28, 28)

In [50]:
# Test ELBO
log_qz_x, z_samples, qz_x = encoder(x)

log_px_z = decoder(x, z_samples)

In [52]:
qz_x

Normal(loc: torch.Size([32, 20]), scale: torch.Size([32, 20]))

In [69]:
(kl_divergence(qz_x, Normal(0., 1.)) == \
    kl_divergence(qz_x, Normal(torch.zeros_like(z_samples), torch.ones_like(z_samples)))).all()

tensor(True)

In [63]:
kl_divergence(qz_x, Normal(torch.zeros_like(z_samples), torch.ones_like(z_samples)))

tensor([[6.4744e-02, 1.1422e-01, 7.7190e-02, 4.9406e-02, 8.0068e-03, 1.1207e-01,
         2.9643e-03, 3.7173e-01, 8.6850e-02, 4.2416e-02, 1.0440e-02, 1.2173e-01,
         1.6011e-02, 7.9263e-02, 3.5556e-02, 9.7776e-02, 2.0983e-02, 5.9932e-03,
         1.2641e-01, 9.0579e-02],
        [3.7803e-01, 2.0156e-03, 2.9930e-02, 4.3775e-02, 2.7872e-01, 2.1063e-01,
         1.8791e-01, 2.5513e-01, 1.0867e-01, 1.1949e-01, 9.3072e-02, 6.3671e-01,
         4.0277e-02, 8.5223e-02, 7.2289e-02, 5.1981e-02, 1.3831e-01, 3.8681e-01,
         7.7657e-02, 4.3821e-02],
        [1.3451e-01, 3.9973e-02, 3.4743e-01, 2.8815e-02, 9.0153e-02, 2.0260e-02,
         9.6283e-02, 4.0331e-01, 2.6368e-02, 3.1427e-01, 2.5026e-01, 1.5003e-01,
         2.0004e-02, 5.7163e-02, 3.5995e-02, 7.5412e-02, 4.3573e-02, 6.3582e-02,
         4.2435e-02, 4.1569e-02],
        [3.8111e-01, 8.2337e-03, 1.0662e-01, 1.0878e-01, 4.7289e-03, 1.1748e-02,
         3.0151e-02, 1.0167e-01, 8.7260e-02, 1.1628e-02, 1.4355e-01, 4.8033e-02,
       

In [70]:
kld = kl_divergence(qz_x, Normal(0, 1)).sum(-1)